## Generating tasks

There are several functions that are relevant for parallel computing in Julia

- `t = remotecall(f, nth_process, args...; kwargs...) ` allows us to call `f` into process `nth_process`.

- `fetch(t)` allows us to recumerate in the main process the result of the task t.

In [2]:
?remotecall

search: remotecall remotecall_wait remotecall_fetch RemoteChannel



```
remotecall(f, id::Integer, args...; kwargs...) -> Future
```

Call a function `f` asynchronously on the given arguments on the specified process. Returns a [`Future`](@ref). Keyword arguments, if any, are passed through to `f`.

```
remotecall(f, pool::AbstractWorkerPool, args...; kwargs...) -> Future
```

`WorkerPool` variant of `remotecall(f, pid, ....)`. Waits for and takes a free worker from `pool` and performs a `remotecall` on it.


### `remotecall` and `fetch` functions

The `remotecall` function allows us send a function to a certain thread ad execute it there. Then we can use the function `fetch` to retrieve the result of the function.

What `remotecall` does is to generate a `Future` object.

A `Future` object is a subtype of `Base.Distributed.AbstractRemoteRef` that allows us to identify some task to a process.

- `Base.Distributed.AbstractRemoteRef` has 2 subtypes
    - `Future`
    - `RemoteChannel`
    
    
    
```julia
id = 2 # worker/process id
task = remotecall(rand,id,(5,3))
result = fetch(task)
```
    
    
##### `remotecall` function and `@spawnat` macro

We can use the `@spawnat` macro to execute some function to a certain process. This macro works as follows:

```julia
id = 2 # worker/process id
t = @spawnat id rand(10) .+ 2
result = fetch(t)
```


In [186]:
println(supertype(Future))
println(subtypes(Base.Distributed.AbstractRemoteRef))

Base.Distributed.AbstractRemoteRef
Union{DataType, UnionAll}[Future, RemoteChannel]


In [187]:
task = remotecall(rand,1,(5,3))

Future(1, 1, 75, Nullable{Any}())

we can get the result of the task doing `fetch(task)`

In [188]:
fetch(task)

5×3 Array{Float64,2}:
 0.276478    0.24084   0.237455
 0.990272    0.621523  0.865307
 0.595366    0.649491  0.741228
 0.00757128  0.479466  0.692141
 0.629508    0.143286  0.949581

We can specify the worker that we want to assign the remotecall with its id.

Now let us generate a 2x10 matrix in worker 2.

In [189]:
addprocs()

8-element Array{Int64,1}:
 10
 11
 12
 13
 14
 15
 16
 17

In [190]:
task2 = remotecall(rand,2,(2,10))

Future(2, 1, 84, Nullable{Any}())

In [191]:
fetch(task2)

2×10 Array{Float64,2}:
 0.833629  0.87937   0.677581  0.23421   …  0.746736  0.0192053  0.301025
 0.991665  0.958105  0.74373   0.129838     0.514981  0.66606    0.463229

### **`remotecall_fetch`** function 

If we want to get the result of the function that we run using `remotecall`, instead of using `remotecall` and `fetch` we can do a single call to the function **`remotecall_fetch`**.

In [192]:
?remotecall_fetch

search: remotecall_fetch remotecall_wait



```
remotecall_fetch(f, id::Integer, args...; kwargs...)
```

Perform `fetch(remotecall(...))` in one message. Keyword arguments, if any, are passed through to `f`. Any remote exceptions are captured in a [`RemoteException`](@ref) and thrown.

See also [`fetch`](@ref) and [`remotecall`](@ref).

```
remotecall_fetch(f, pool::AbstractWorkerPool, args...; kwargs...) -> result
```

`WorkerPool` variant of `remotecall_fetch(f, pid, ....)`. Waits for and takes a free worker from `pool` and performs a `remotecall_fetch` on it.


Notice that sending a function to a worker, waiting the worker to finisht the execution of the function and returning the result to the main worker (or process) takes time. It is not trivial to take advantage of this function.

In [193]:
@time remotecall_fetch(rand,2,(2,10))

  0.000333 seconds (97 allocations: 2.969 KiB)


2×10 Array{Float64,2}:
 0.0861554  0.706942  0.898549  0.940014  …  0.613416  0.914992   0.512348
 0.176484   0.271358  0.770157  0.575554     0.945261  0.0519798  0.662977

In [194]:
@time rand(2,10)

  0.000006 seconds (5 allocations: 400 bytes)


2×10 Array{Float64,2}:
 0.479556  0.44672   0.0540984  0.620533  …  0.7005    0.868207  0.498375
 0.128198  0.640125  0.718999   0.990833     0.143077  0.730378  0.544621

## Moving Data between processess

Usually we will want to do something more complex than to create an array in a process and send it to the main process. 

#### `@spawn` macro and `Distributed.spawn_somewhere`  function.

Sometimes we don't need (or want) to explicitly control which worker will execute a task. In the case of `remotecall(f, id, args)` and `remotecall_fetch(f, id, args)` we had to explicitly tell the functions the `id` of the worker that will take the job. 

A simple way to execute a function in a different process than the main process, without any need to control the `id` of the worker, is to use the `@spawn` macro. This macro will execute an expression in an available worker.

The `@spawn` macro ends up using the `Distributed.spawn_somewhere` function.



In [195]:
?@spawn

```
@spawn
```

Creates a closure around an expression and runs it on an automatically-chosen process, returning a [`Future`](@ref) to the result.


In [196]:
m = rand(1000,1000);

In [197]:
squared = @spawn m^2

Future(2, 1, 87, Nullable{Any}())

In [198]:
fetch(squared)

1000×1000 Array{Float64,2}:
 242.025  242.69   244.783  248.33   …  248.823  242.285  246.893  240.034
 251.233  250.4    257.441  250.633     257.972  252.298  253.505  253.985
 246.157  243.957  251.767  251.724     254.085  248.012  253.429  248.421
 239.912  246.906  250.325  255.908     247.527  245.938  249.477  247.999
 246.22   250.468  260.711  262.508     258.827  253.909  251.05   255.893
 242.225  245.557  251.492  246.47   …  249.55   241.657  247.301  245.94 
 240.315  239.083  247.634  245.96      244.474  239.432  242.189  242.713
 247.113  243.45   253.68   255.297     255.98   252.688  255.56   253.621
 242.332  238.216  242.819  246.633     247.934  238.251  247.086  244.665
 255.359  255.19   259.751  259.982     256.078  253.759  257.323  254.628
 249.237  244.057  253.351  256.897  …  253.203  252.842  255.884  253.931
 245.469  238.558  242.289  248.61      244.69   235.643  250.532  248.708
 243.894  248.365  257.448  246.811     252.515  247.338  249.915  247.3

In [199]:
fetch(@spawnat 2 m^2)

1000×1000 Array{Float64,2}:
 242.025  242.69   244.783  248.33   …  248.823  242.285  246.893  240.034
 251.233  250.4    257.441  250.633     257.972  252.298  253.505  253.985
 246.157  243.957  251.767  251.724     254.085  248.012  253.429  248.421
 239.912  246.906  250.325  255.908     247.527  245.938  249.477  247.999
 246.22   250.468  260.711  262.508     258.827  253.909  251.05   255.893
 242.225  245.557  251.492  246.47   …  249.55   241.657  247.301  245.94 
 240.315  239.083  247.634  245.96      244.474  239.432  242.189  242.713
 247.113  243.45   253.68   255.297     255.98   252.688  255.56   253.621
 242.332  238.216  242.819  246.633     247.934  238.251  247.086  244.665
 255.359  255.19   259.751  259.982     256.078  253.759  257.323  254.628
 249.237  244.057  253.351  256.897  …  253.203  252.842  255.884  253.931
 245.469  238.558  242.289  248.61      244.69   235.643  250.532  248.708
 243.894  248.365  257.448  246.811     252.515  247.338  249.915  247.3

#### What `@spawn` does internally

spawn_somewhere(thunk) = spawnat(nextproc(),thunk)


In [200]:
@macroexpand @spawn m^2

:((Base.Distributed.spawn_somewhere)((()->begin  # distributed/macros.jl, line 20:
                m ^ 2
            end)))

In [201]:
?Distributed.spawn_somewhere

No documentation found.

`Base.Distributed.spawn_somewhere` is a `Function`.

```
# 1 method for generic function "spawn_somewhere":
spawn_somewhere(thunk) in Base.Distributed at distributed/macros.jl:17
```


In [202]:
aux = Distributed.spawn_somewhere(x->x^2(m))

Future(3, 1, 91, Nullable{Any}())

In [205]:
# Why is this not working??
fetch(aux)

LoadError: [91mOn worker 3:
[91mMethodError: no method matching (::Base.Serializer.__deserialized_types__.##71#72)()[0m
Closest candidates are:
  #71([91m::Any[39m) at In[202]:1[39m
#103 at ./distributed/process_messages.jl:264 [inlined]
run_work_thunk at ./distributed/process_messages.jl:56
run_work_thunk at ./distributed/process_messages.jl:65 [inlined]
#96 at ./event.jl:73[39m

In [206]:
@which Distributed.spawn_somewhere(x-> x^2)

spawn_somewhere(thunk) in Base.Distributed at distributed/macros.jl:17

In [209]:
nworkers()

16

In [213]:
nprocs()

17

#### Understanding where data is 

In [217]:
a=@spawn randn(5,5)^2 #This is allocated into another process
fetch(a)

5×5 Array{Float64,2}:
  1.46853    -1.53682    1.30386   2.31497  -0.561842
  3.52336    -4.04827   -3.8246    3.35829   2.16229 
 -0.653903    1.43576   -3.12595   5.70666  -2.80629 
 -3.53757     0.727602   2.22825   3.01629  -1.86741 
  0.0674948   3.86534   -3.44     -1.11669  -1.41904 

In [218]:
b=rand(5,5) # this is created in main process, send to another and the result fetched back
task = @spawn b^2
fetch(task)

5×5 Array{Float64,2}:
 1.50994   1.42094   1.01669   1.5138    1.19659 
 1.46354   1.08328   0.781699  1.20854   0.885973
 0.980027  0.670013  0.899145  0.872088  0.649319
 2.15017   1.85373   1.46218   1.81961   1.0685  
 1.59937   1.32017   1.06774   1.2648    1.21548 

### Everywhere macro

In [212]:
@macroexpand @everywhere m^2

quote  # distributed/macros.jl, line 99:
    nothing # distributed/macros.jl, line 100:
    (Base.Distributed.sync_begin)() # distributed/macros.jl, line 101:
    for #81#pid = (Base.Distributed.workers)() # distributed/macros.jl, line 102:
        (Base.Distributed.async_run_thunk)((()->begin  # distributed/macros.jl, line 102:
                    (Base.Distributed.remotecall_fetch)(Base.Distributed.eval_ew_expr, #81#pid, $(Expr(:copyast, :($(QuoteNode(:(m ^ 2)))))))
                end)) # distributed/macros.jl, line 103:
        (Base.Distributed.yield)()
    end # distributed/macros.jl, line 108:
    if (Base.Distributed.nprocs)() > 1 # distributed/macros.jl, line 109:
        (Base.Distributed.async_run_thunk)((()->begin  # distributed/macros.jl, line 109:
                    (Base.Distributed.eval_ew_expr)($(Expr(:copyast, :($(QuoteNode(:(m ^ 2)))))))
                end))
    end # distributed/macros.jl, line 112:
    (Base.Distributed.sync_end)()
end